# Processing the data

## Introduction

**Learning Objectives**

- What is a protein language model
- Getting raw protein data ready
- Protein sequence processing
- Vectorise protein sequence with protein language model

The large part of this section will be a dive into one of the most crucial aspects of machine learning workflows; data preprocessing. 

Specifically, we will focus on handling and preparing protein sequence data retrieved from the [**RefSeq** database](https://www.ncbi.nlm.nih.gov/refseq/). Proper preprocessing can significantly enhance the performance of machine learning models by ensuring that the data is clean, well-structured, and suitably formatted for training.

Our goal in this tutorial is to equip you with the tools and techniques necessary to transform raw protein sequences into a format that is compatible with machine learning models utilising protein language models.

## What is a protein language model?

In traditional methods, when aiming to extract features from proteins, common approaches include bioinformatics techniques such as calculating evolutionary scores (i.e., conserved scores) based on homology sequences, or assessing overall physicochemical properties of the protein, like *hydrophobicity* or *hydrophilicity*. However, human understanding of proteins is limited, which in turn limits the range of extractable features.

In 2021, Meta and Rost's group, utilizing 250 million protein sequences, developed [ESM](https://github.com/facebookresearch/esm) and [ProtT5](https://github.com/agemagician/ProtTrans) models, collectively known as protein language models, each with their own model architecture. 

Unlike a standard neural network, language models are designed to understand and generate sequential data by learning patterns and relationships within sequences. Rather than simply mapping inputs to outputs through fixed layers, language models use specialised architectures called **Transformers**, which convert sequences into a format that can be processed by a computer, *embeddings* (more on this later). The embeddings are then processed by the **attention mechanism**, which allows the model to identify which parts of the sequence are most relevant to the task at hand. The **attention mechanism** is able to understand the sequence as a whole, capturing long-range dependencies between elements, rather than processing it piece by piece, which was standard in previous natural language processing methods such as *recurrent neural networks*, *hidden Markov models*, and so on.

Information about protein sequences is learned during extensive training on very large datasets. For the **ESM** protein models we will be using, during training, a portion of the amino acids in the input sequences was randomly masked; for example, the protein sequence "AOPIEYTLKQGFMCV" might be altered to "AO`<mask>`IEYTLKQGF`<mask>`CV". The task for the model was to predict the masked amino acids based on the unmasked content.

This process is similar to how Large Language Models (your ChatGPTs, Claude, Gemini, etc.) work, but instead of amino acids, words are used. Any problem which involves sequence data can potentially be solved with a language model.

Whilst this lesson will focus on protein language models, the principles are the same for any sequence data. If there are no pre-existing language models for a specific sequence data, it is possible to design and train your own language model. However, due to the need for large amounts of data and processing power this complex task is beyond the scope of this lesson.

![Protein to Neural Network](./images/study-evaluates-deep-l.jpg)

NOTE BOX

If you wish to learn more about protein models in depth, please read the information [here](https://www.apoorva-srinivasan.com/plms/#:~:text=Protein%20%E2%80%9Clanguage%E2%80%9D%20is%20a%20lot,other%20domains%2C%20with%20profound%20implications.).

Addtionally, for a deeper understanding of transformer architecture and attention mechanism, you can explore this [article](https://towardsdatascience.com/transformers-141e32e69591). 

## Data Preparation

Before we get to the language model we need to process our data. 

This lesson was created by the author of the [VariPred](https://github.com/wlin16/VariPred) toolbox. The toolbox makes use of pre-trained protein language models to predict variant pathogenicity. More can be read about the toolbox in the Nature article [here](https://www.nature.com/articles/s41598-024-51489-7).

The data used in this lesson can be downloaded from the repository itself, however it is also available in the *data* folder in your lesson repository for you to run yourselves.

### Loading the dataset

In [ ]:
import pandas as pd

In [ ]:
seq = pd.read_csv("./data/transcript_seq.csv")
train = pd.read_csv("./data/VariPred_train.txt", sep="\t")
test = pd.read_csv("./data/VariPred_test.txt", sep="\t")

Our data comes in **three** parts. `seq` contains the amino acid sequences of the proteins, plus a column with a protein ID, `NP_id`.

In [ ]:
print(seq.head(3))

The remaining **two** dataframes are for the training data (`train`) and the other is for the validation data (`test`). These dataframes contain the information about mutations of the wildtype sequence above.

In [ ]:
print(train.head(3))

In [ ]:
print(test.head(3))

Here, target_id column in both train and test dataframes contains information in the format of "`RefSeq id`: **wildtype amino acid** at a **specific position** mutated to **mutated amino acid**". The label column indicates whether the missense mutation is **benign (0)** or **pathogenic (1)**.

## Regular Expression

The `seq` dataframe contains the protein id (here all be represented with NP codes) of the wildtype and the corresponed amino acid sequence. The `train` or `test` dataset contains all necessary details of each variant, including protein id (NP code), wildtype amino acid, mutational position, and mutated amino acid, which are integrated in `HGVSp` format in the target_id column.

In this section, we aim to
1. parse the information of each variant in the `HGVSp` format, and
2. generate mutated protein sequences according to the mutational information and the corresponded wildtype sequence

To do so, we need to seperate information of each variant and stored them individually into new columns in the datasets. Here, we can apply a super useful strategy called **regular expression**. Whilst this technique is not directly related to protein language models, it is a powerful tool that can be applied to a wide range of tasks in bioinformatics and data analysis.

**Regular expression** (shortened as *regex* ), is a sequence of characters that specifies a match pattern in text. Usually such patterns are used by string-searching algorithms to "find" or "find and replace" subsequences within a string. We won't go over creating a **regex** seqeunce here, just its application. However, the tip box below has a useful workflow for creating one.

TIP BOX

A great website that can help you to extract sequence of characters that specifies a match pattern in text is [https://regex101.com/](https://regex101.com/)

The easiest way to get a correct regular expression pattern for your target string is:

1. Copy and paste the string you would like to match the pattern into the `TEST STRING` blank in https://regex101.com/. Here, for example, NP_000005.2:p.Arg1031Gln

2. Then, go to your favourite genAI LLM, and use this prompt: 

```
What is the regular expression pattern for strings like "NP_000005.2:p.Arg1031Gln" in python? I want to match the first item before colon, the second item with three characters after p., all following digitals, and the last three characters in different groups. For example, here I would like to get "NP_000005.2", "Arg", "1031", and "Gln"
```

3. Copy and paste the regular expression pattern given by ChatGPT into the blank showing **insert your regular expression here** at https://regex101.com/

### Applying regex

If we look at one of our **target IDs** again, it is made up of several parts delineated by `.` or `:`, with changing sections of letters and numbers.

In [ ]:
test["target_id"][0]

In [ ]:
#'re' is a module in Python that provides support for regular expressions. 
import re

The regex pattern below has been designed to parse and extract specific compenents from our target ID, breaking it down into **five** known groups.

In [ ]:
regex_pattern = r"(.+(?=:))(:p.)([A-Z][a-z]+)([0-9]+)([A-Z][a-z]+)"

`re.search` is a function in the 're' module that searches for a specified pattern within a string. It returns a match object if the pattern is found, otherwise it returns None.

In [ ]:
txt = re.search(regex_pattern, test.target_id[0])

print(txt)
print("-" * 50)
print("input content: ", txt[0])
print("first match item: ", txt[1])
print("second match item: ", txt[2])
print("third match item: ", txt[3])
print("fourth match item: ", txt[4])
print("fifth match item: ", txt[5])

The regex pattern has found the **five** items. We are only interested in **four** of the items:
- protein id: `txt[1]`
- wildtype amino acid: `txt[3]`
- mutational position: `txt[4]`
- mutated amino acid: `txt[5]`

To extract them we will create a *function*:

In [ ]:
def get_id(input_id):
    """
    Parse protein variant notation to extract gene identifier and mutation details.
    
    This function uses regular expressions to parse HGVS-style protein variant 
    notation (e.g., "BRCA1:p.Gly123Ser") and extract the gene identifier, 
    wild-type amino acid, position, and mutant amino acid.
    
    Parameters
    ----------
    input_id : str
        Protein variant identifier in HGVS format. Expected format:
        ``<gene_id>:p.<WtAA><position><MtAA>``
        
        Where:
        - ``<gene_id>`` : Gene or protein name
        - ``:p.`` : Protein-level variant prefix
        - ``<WtAA>`` : Wild-type amino acid (three-letter code, e.g., "Gly")
        - ``<position>`` : Amino acid position in the sequence (integer)
        - ``<MtAA>`` : Mutant amino acid (three-letter code, e.g., "Ser")
    
    Returns
    -------
    gene_id : str
        Gene or protein identifier extracted from the input string.
    aa_index : str
        Position of the amino acid mutation in the protein sequence (as string).
    wt : str
        Wild-type amino acid in three-letter code (e.g., "Gly", "Ala").
    mt : str
        Mutant amino acid in three-letter code (e.g., "Ser", "Val").
  
    """
    
    # Use regular expression to match and extract specific parts of the input_id
    txt = re.search(r"(.+(?=:))(:p.)([A-Z][a-z]+)([0-9]+)([A-Z][a-z]+)", input_id)
    
    # Extract the gene_id, wildtype amino acid, mutational position, and mutated amino acid from the matched groups
    gene_id = txt.group(1)
    wt = txt.group(3)
    aa_index = txt.group(4)
    mt = txt.group(5)

    return gene_id, aa_index, wt, mt

Rather than looping through every sample in the dataframe, we can use the **pandas** method [`apply`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html), which will "apply" the given function to the value of each row. This is much faster the looping through the rows as it is vecotrised.

In [ ]:
test['target_id'].apply(get_id)[0:3]

In [ ]:
type(test['target_id'].apply(get_id))

The apply method returns a *pandas series* with each item being a tuple containing each of the returned variables. This can be *unpacked* and *zipped* so that it can added to **four** new columns in one go.

In [ ]:
test['gene_id'], test['aa_index'], test['wt_aa'], test['mt_aa'] = zip(*test['target_id'].apply(get_id))

print(test.head(3))

Checking the updated dataframe the ID, mutational position, wildtype amino acid and mutated amino acid have been added to the four specified columns.

Given it works, lets apply it to the training dataset too.

In [ ]:
train['gene_id'], train['aa_index'], train['wt_aa'], train['mt_aa'] = zip(*train['target_id'].apply(get_id))

print(train.head(3))

## AA Code Conversion

The next step is to create a **mutated sequence** from the wildtype sequence stored in the `seq` dataframe, according to the mutational information.

However, the amino acids in the dataframes are all in three letter codes, e.g. *Gly*, while those in the amino acid sequences are all in one letter codes, e.g. G.

If we use the amino acids in a three letter representation, the generated mutated protein sequences will be wrong, e.g. sequence "AIOPC" with mutational information "Ala1Cys" (first amino acid being mutated from Alanine into Cysteine) will become "CysIOPC".

Therefore, before we generate mutated protein sequences, we need to convert `wt_aa` and `mt_aa` from three letter codes into one letter codes

### Application

This can be done simply by mapping a *dictionary* to the given column. Where the dictionary contains as its **keys** the three letter code and the **values** the equivalent single letter code.

In [ ]:
# Dict to convert amino_acid abbreviations into amino_acid_codes
amino_acid_dict = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

# When working with a singular column use map
def convert_code(data, aa_dict):
     """00a_introduction.ipynb"""

     # First ensure all the letters are capitalised 
     # For this we map a lambda function that applies the string method .upper()
     data['wt_aa'] = data['wt_aa'].map(lambda x: x.upper())
     data['mt_aa'] = data['mt_aa'].map(lambda x: x.upper())

     # Map the dictionary to the given columns
     data['wt_aa'] = data['wt_aa'].map(aa_dict)
     data['mt_aa'] = data['mt_aa'].map(aa_dict)


# Call the function
convert_code(train, amino_acid_dict)
convert_code(test, amino_acid_dict)

print("Train dataset:")
print(train.head(3), "\n")
print("Train dataset:")
print(test.head(3))

## Merging the data

Having successfully converted all amino acids into one-letter codes, we are now ready to create **mutated sequences** for each protein based on the corresponding wildtype sequence and mutation data.

To begin, we must **merge** the dataframe containing the wildtype sequence with the dataframe containing mutation information:

In [ ]:
def merge_seq_with_mutational_info(seq_df, input_df):
    """
    Merge sequence data with mutational information based on gene identifiers.

    Parameters
    ----------
    seq_df : pandas.DataFrame
        Dataframe containing protein sequence information. Must include:
        
        - ``gene_id`` : str
            Gene or protein identifier (merge key)
        - ``sequence`` : str
            Protein sequence (will be renamed to 'wt_seq')
        - Additional columns as needed
    
    input_df : pandas.DataFrame
        Dataframe containing mutational and pathogenicity information. Must include:
        
        - ``gene_id`` : str
            Gene or protein identifier (merge key)
        - ``pathogenicity`` : any
            Pathogenicity classification (will be renamed to 'label')
        - Additional mutation-related columns (e.g., position, amino acid changes)
    
    Returns
    -------
    pandas.DataFrame
        Merged dataframe with standardised column names
    """

    # Merge two dataframes based on the 'gene_id' column using a right join and drop rows with missing values
    output_df = pd.merge(seq_df, input_df, on='gene_id', how='right').dropna(axis=0)
    
    # Rename columns 'sequence' to 'wt_seq' and 'pathogenicity' to 'label'
    output_df.rename(columns={'sequence': 'wt_seq', 'pathogenicity': 'label'}, inplace=True)
    
    return output_df

In [ ]:
merged_test = merge_seq_with_mutational_info(seq, test)

This error is a common one for anyone looking to merge two dataframes into one. The error details that the column "gene_id" cannot be the key column to merge two dataframe. This is becausethe **merge** function needs the dataframes to share a column of the same name. However, the column with protein ids in `seq` dataframe is named as `NP_id`, while that in the `test` or `train` dataframe is named as `gene_id`.

In this case, we need to rename the column name in the `seq` dataframe for ease.

In [ ]:
seq.rename(columns={'NP_id': "gene_id"}, inplace = True)

print(seq.head(3))

In [ ]:
merged_test = merge_seq_with_mutational_info(seq, test)

print(merged_test.head(3))

In [ ]:
merged_train = merge_seq_with_mutational_info(seq, train)

print(merged_train.head(3))

## Generating Mutated Seqeunces

With the merged dataframe, we can finally generate mutated sequences according to the mutational information: `aa_index` and `mt_aa`.

The function below will generate the mutation specified in the training and test datset.

In [ ]:
def create_mt_sequence(df):
    """
    Create mutated protein sequences based on variant information in a dataframe.
    
    This function takes a dataframe containing wild-type sequences and mutation 
    information, generates the corresponding mutated sequences, and adds them 
    along with sequence length information to the dataframe.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe containing mutation information. Must include the following columns:
        
        - ``wt_seq`` : str
            Wild-type (original) protein sequence
        - ``aa_index`` : int or str
            1-based position of the amino acid to mutate
        - ``mt_aa`` : str
            Single-letter code of the mutant amino acid
        - ``target_id`` : str
            protein identifier
        - ``wt_aa`` : str
            Single-letter code of the wild-type amino acid
        - ``label`` : any
            Label or classification for the variant
    
    Returns
    -------
    pandas.DataFrame
        Modified dataframe with added columns and reordered structure
    
    """

    # Initialise empty lists to store mutated sequences and lengths
    mt_sequence = []
    length = []
    
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Calculate the length of the wild-type sequence
        seq_len = len(row['wt_seq'])
        
        # Store the wild-type sequence in a variable
        string = row['wt_seq']
        
        # Calculate the position to mutate (adjusting for 0-based indexing)
        posn = int(row['aa_index']) - 1
        
        # Get the new amino acid to replace at the mutation position
        new_aa = row['mt_aa']
        
        # Create the mutated sequence by replacing the amino acid at the mutation position
        result = string[:posn] + new_aa + string[posn+1:]
        
        # Append the length of the wild-type sequence and the mutated sequence to the respective lists
        length.append(seq_len)
        mt_sequence.append(result)
  
    df['mt_seq'] = mt_sequence
    df['Length'] = length
    df['Length'] = df['Length'].apply(lambda x: int(x))
    df['aa_index'] = df['aa_index'].apply(lambda x: int(x))

    column_order = ['target_id','aa_index','Length','wt_aa','mt_aa','wt_seq','mt_seq','label']
    df = df[column_order]

    return df

In [ ]:
train_final = create_mt_sequence(merged_train)
print("Train mutations:")
print(train_final.head(3), "\n")

test_final = create_mt_sequence(merged_test)
print("Test mutations:")
print(test_final.head(3))

Calling the function on both the *train* and *test* dataset generates our enriched dataset. These can be saved as a **csv** for use in subseqeunt sections.

In [ ]:
train_final.to_csv("./data/processed_train.csv", index=False)
test_final.to_csv("./data/processed_test.csv", index=False)

## Summary

We are now at the end of our data pre-processing step and are ready to move on to the featurisation step in the next section using a protein language model. This step will involve leveraging one of the most powerful protein language models, **ESM1b**, to extract meaningful features from the processed protein sequence data.